In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.7-cp311-cp311-linux_x86_64.whl size=4552804 sha256=27e5038d8cbe04bf1de3d78684dbec2e27b75666e1e3be1c81a39e3989bcd1d7
  Stored in directory: /root/.cache/pip/wheels/eb/82/79/ac77fcd49324b75ae6aa18e63a87cf9da4371a57e2cdc8dc03
Successfully built llama-cpp-python


In [ ]:
!pip install --upgrade python-telegram-bot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.5/669.5 kB 31.9 MB/s eta 0:00:00


In [ ]:
import os
MODEL_PATH = "/content/drive/MyDrive/mistral-7b-instruct-v0.1.Q4_K_M.gguf"

if os.path.exists(MODEL_PATH):
    print("Model file found ✅")
else:
    print("Model file NOT found ❌")

Model file found ✅


In [ ]:
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters
from llama_cpp import Llama
import os
import asyncio

# Load LLaMA model
llm = Llama(model_path=MODEL_PATH, n_ctx=512, n_threads=4)


# Telegram Bot Token (Replace with your token)
TELEGRAM_TOKEN = "7733772030:AAFPROxpal65SDkiNDuevN0_7CNQAwKFZ-Q"

async def start(update: Update, context):
    await update.message.reply_text("Hello! I am your LLaMA-powered chatbot.")

async def handle_message(update: Update, context):
    user_input = update.message.text
    response = llm(user_input, max_tokens=300)["choices"][0]["text"].strip()
    await update.message.reply_text(response)




llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/drive/MyDrive/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:    

In [ ]:
import nest_asyncio
import asyncio


In [ ]:

# ✅ Fix for Colab's async loop issue
nest_asyncio.apply()

# Start the bot
def main():
    """Start the bot."""
    application = ApplicationBuilder().token("7733772030:AAFPROxpal65SDkiNDuevN0_7CNQAwKFZ-Q").build()

    # Add handlers
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    print("Bot is running...")

    asyncio.get_event_loop().run_until_complete(application.run_polling())

if __name__ == "__main__":
    main()


Bot is running...


llama_perf_context_print:        load time =    3836.55 ms
llama_perf_context_print: prompt eval time =    3836.37 ms /     9 tokens (  426.26 ms per token,     2.35 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    3837.12 ms /    10 tokens
ERROR:telegram.ext.Application:No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_application.py", line 1325, in process_update
    await coroutine
  File "/usr/local/lib/python3.11/dist-packages/telegram/ext/_handlers/basehandler.py", line 158, in handle_update
    return await self.callback(update, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-5-d9c6d57b0258>", line 20, in handle_message
    await update.message.reply_text(response)
  File "/usr/local/lib/python3.11/dist-pack

RuntimeError: Cannot close a running event loop

#Improved codes:

In [ ]:
chat_history = []  # Global variable to store past interactions
SYSTEM_PROMPT = """You are a helpful but concise AI assistant.
Reply in a maximum of 2 sentences. Do not ask follow-up questions unless directly requested."""

async def handle_message(update: Update, context):
    user_input = update.message.text

    if len(chat_history) > 5:
        chat_history.pop(0)

    chat_history.append(f"User: {user_input}")

    # Include system instructions
    prompt = f"{SYSTEM_PROMPT}\n\n" + "\n".join(chat_history) + "\nAssistant:"

    response = llm(
        prompt,
        max_tokens=80,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        stop=["\nUser:", "Assistant:"]
    )["choices"][0]["text"].strip()

    chat_history.append(f"Assistant: {response}")

    await update.message.reply_text(response)

In [ ]:
async def start(update: Update, context):
    await update.message.reply_text("Hello! I am your chatbot. Ask me anything!")

async def reset(update: Update, context):
    global chat_history
    chat_history = []  # Clear chat history
    await update.message.reply_text("Chat history cleared. Let's start fresh!")

def main():
    application = ApplicationBuilder().token(TELEGRAM_TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(CommandHandler("reset", reset))  # New reset command
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    print("Bot is running...")

    asyncio.get_event_loop().run_until_complete(application.run_polling())

if __name__ == "__main__":
    main()


Bot is running...


Llama.generate: 1 prefix-match hit, remaining 45 prompt tokens to eval
llama_perf_context_print:        load time =    3836.55 ms
llama_perf_context_print: prompt eval time =   21453.61 ms /    45 tokens (  476.75 ms per token,     2.10 tokens per second)
llama_perf_context_print:        eval time =   24792.43 ms /    39 runs   (  635.70 ms per token,     1.57 tokens per second)
llama_perf_context_print:       total time =   46270.37 ms /    84 tokens
Llama.generate: 85 prefix-match hit, remaining 15 prompt tokens to eval
llama_perf_context_print:        load time =    3836.55 ms
llama_perf_context_print: prompt eval time =    8002.40 ms /    15 tokens (  533.49 ms per token,     1.87 tokens per second)
llama_perf_context_print:        eval time =   15184.59 ms /    24 runs   (  632.69 ms per token,     1.58 tokens per second)
llama_perf_context_print:       total time =   23201.42 ms /    39 tokens
Llama.generate: 124 prefix-match hit, remaining 11 prompt tokens to eval
llama_perf_con